In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]
import seaborn as sns

import netCDF4 as nc
import xarray as xr

from src.netcdftools import *
from src.models import Fitter
from sklearn.linear_model import LinearRegression
import glob

In [6]:
thick = SatelliteNC(path='/Users/drigo/ITMO/_disser/surrogate/data/raw/sattelite/ice_thick_y2016.nc',
                variable='Thickness')
conc = SatelliteNC(path='/Users/drigo/ITMO/_disser/surrogate/data/raw/sattelite/conc_y2016.nc',
               variable='ice_conc')